The goal of this CNN is to classify wind faults into four categories: 
1. 3 Phase to Ground
2. 2 Phase line to line
3. 2 phase to earth
4. 1 pole line to ground. 



In [25]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torchviz import make_dot
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


Preprocessing + Rolling Windows

In [26]:
WINDOW_SIZE = 20  # 1 second window (adjust if needed)
OVERLAP_SIZE = 15
LABEL_MAPPING = {
    "3P": 4,  # 3-phase to ground
    "2P": 3,   # 2-phase line to line
    "2P.EARTH": 2, # 2-phase to earth
    "1P": 1,    # 1-pole line to ground
    "normal": 0  # Normal
}
class WindFaultDataset(Dataset):
    """ PyTorch Dataset for Wind Fault Classification """
    def __init__(self, file_paths):
        self.data = []
        self.labels = []
        self.scaler = MinMaxScaler()

        for file_path in file_paths:
            features, label = self.process_csv(file_path)
            windows, labels = self.create_rolling_windows(features, label, WINDOW_SIZE, OVERLAP_SIZE)
            self.data.append(windows)
            self.labels.append(labels)

         # Check if all windows have the same number of columns
        num_columns = [window.shape[2] for window in self.data]
        if len(set(num_columns)) != 1:
            raise ValueError(f"Mismatch in number of columns: {num_columns}")

        self.data = np.vstack(self.data)  # Stack all windows
        self.labels = np.hstack(self.labels)

    def process_csv(self, file_path):
        """ Load and preprocess a CSV file. """
        df = pd.read_csv(file_path)
  
        df = df.drop(df.columns[0], axis=1)  # Dropping time column
        df = df.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
        
        # Extract label from filename
        fault_type = os.path.basename(file_path).split("-")[0]
        label = LABEL_MAPPING.get(fault_type, 0)
        
        # Normalize features
        feature_data = self.scaler.fit_transform(df)
        # Add label column to feature_data
        label_column = np.full((feature_data.shape[0], 1), label)
        features = np.hstack((feature_data, label_column))
        
        return features, label

    def create_rolling_windows(self, data, label, window_size, overlap_size):
        #""" Generate overlapping rolling windows of data. """
        windows = []
        labels = []
        step_size = window_size - overlap_size
        for i in range(0, len(data) - window_size + 1, step_size):  # Overlapping windows
            windows.append(data[i:i + window_size])
            labels.append(label)
        return np.array(windows), np.array(labels)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

I am going to make a seperate folder within Grid_faults to store the noise generated data. 

In [27]:
training_data = 'Dataset1'
test_data = 'Distribution_faults'
train_paths = []
test_paths = []
for file in os.listdir(training_data):
    train_paths.append(os.path.join(training_data, file))
for file in os.listdir(test_data):
    test_paths.append(os.path.join(test_data, file))

train_dataset = WindFaultDataset(train_paths)
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataset = WindFaultDataset(test_paths)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)


In [28]:


# Print dataset shape
sample_data, sample_label = dataset[0]
print("Sample data shape:", sample_data.shape)  # (WINDOW_SIZE, num_features)
print("Sample label:", sample_label)
unique, counts = np.unique(dataset.labels, return_counts=True)
label_stats = dict(zip(unique, counts))
print("Label statistics:", label_stats)
#print(sample_data)

t_np = sample_data.numpy() #convert to Numpy array
df = pd.DataFrame(t_np) #convert to a dataframe
df.to_csv("testfile",index=False) #save to file

Sample data shape: torch.Size([20, 19])
Sample label: tensor(1)
Label statistics: {np.int64(0): np.int64(364), np.int64(1): np.int64(372)}


In [29]:

class WindFaultCNN(nn.Module):
    def __init__(self, input_channels, num_classes, window_size):
        super(WindFaultCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * (WINDOW_SIZE // 2 // 2), 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch, features, time)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

# Get input size
num_features = sample_data.shape[1]  # Number of features (columns)
num_classes = len(LABEL_MAPPING)

# Initialize model
model = WindFaultCNN(input_channels=num_features, num_classes=num_classes, window_size=WINDOW_SIZE)
print(model)

WindFaultCNN(
  (conv1): Conv1d(19, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu): ReLU()
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=160, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)


In [30]:
# model_graph = make_dot(model(sample_data.unsqueeze(0)), params=dict(model.named_parameters()))

# # Render the graph
# model_graph.render("wind_fault_cnn", format="png")

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader)}")

print("Training complete.")


def evaluate_model(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():  # No gradient calculation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted classes
            _, preds = torch.max(outputs, 1)  # Convert softmax to class indices

            # Store results
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, digits=4)
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"Test Loss: {total_loss / len(dataloader):.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", class_report)
    print("\nConfusion Matrix:\n", conf_matrix)

    return accuracy, class_report, conf_matrix

# Example Usage
test_accuracy, test_report, test_conf_matrix = evaluate_model(
    model, test_dataloader, criterion, device
)

Epoch 1, Loss: 1.1941436146562163
Epoch 2, Loss: 0.9158701763765232
Epoch 3, Loss: 0.9059945075093089
Epoch 4, Loss: 0.9055467678888424
Epoch 5, Loss: 0.9050698799861444
Epoch 6, Loss: 0.904959047162855
Epoch 7, Loss: 0.9049222767353058
Epoch 8, Loss: 0.9049022459500545
Epoch 9, Loss: 0.904883273549982
Epoch 10, Loss: 0.9048742958017297
Training complete.
Test Loss: 1.4765
Test Accuracy: 0.4140

Classification Report:
               precision    recall  f1-score   support

           0     0.9977    1.0000    0.9988       862
           1     0.9837    0.9891    0.9864       183
           2     0.9779    0.9973    0.9875      1107
           3     0.1656    0.9167    0.2805      1128
           4     0.8922    0.0988    0.1778      5782

    accuracy                         0.4140      9062
   macro avg     0.8034    0.8004    0.6862      9062
weighted avg     0.8241    0.4140    0.3839      9062


Confusion Matrix:
 [[ 862    0    0    0    0]
 [   2  181    0    0    0]
 [   0    3 